In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url="https://search.danawa.com/dsearch.php?query={}&checkedInfo=N&page={}&limit=40&sort=saveDESC&list=list&maxPrice=200000&tab=main".format("블루투스 헤드폰",1)
browser.get(url=url)

In [ ]:
def get_page(query,page):
  return "https://search.danawa.com/dsearch.php?query={}&checkedInfo=N&page={}&limit=40&sort=saveDESC&list=list&tab=main".format(query,page)

In [ ]:
html = browser.page_source
soup = BeautifulSoup(html,'html.parser')

In [ ]:
# prod_items=soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
# len(prod_items)

In [ ]:
# title = prod_items[0].select('p.prod_name > a')[0].text.strip()
# print(title)

In [ ]:
# spec = prod_items[0].select('div.spec_list')[0].text.strip()
# print(spec)

In [ ]:
# price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
# print(price)

In [ ]:
from tqdm import tqdm
import time

query="블루투스 헤드폰"
total_page=10
data_list=[]
for page in tqdm(range(1,total_page+1)):
  url = get_page(query,page)
  browser.get(url)
  time.sleep(3)
  
  html = browser.page_source
  soup=BeautifulSoup(html,'html.parser')
  prod_items=soup.select('div.main_prodlist > ul.product_list > li.prod_item > div.prod_main_info')
  for item in prod_items:
    title = item.select('p.prod_name > a')[0].text.strip()
    spec = item.select('div.spec_list')[0].text.strip()
    price = item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(",","")
    data_list.append([title,spec,price])
  print(len(data_list))
import pandas as pd
data_df = pd.DataFrame(data_list)
data_df.columns=['product_name','spec','price']
data_df.to_excel('./danawa_crawling_result.xlsx',index=False)

In [3]:
import pandas as pd

data = pd.read_excel('./danawa_crawling_result.xlsx')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  400 non-null    object
 1   spec          400 non-null    object
 2   price         400 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 9.5+ KB


,product_name,spec,price
0,BOSE QC 울트라 헤드폰,유무선헤드폰/블루투스 v5.3/유선연결: 3.5mm/밀폐형/오버이어/음악+통화용/오...,459000
1,APPLE 에어팟 맥스,무선헤드폰/블루투스 v5.0/밀폐형/오버이어/음악+통화용/다이나믹드라이버/오디오코덱...,653070
2,SONY WH-1000XM5,유무선헤드폰/블루투스 v5.2/유선연결: 3.5mm/밀폐형/오버이어/음악+통화용/다...,397790
3,SONY ULT WEAR WH-ULT900N,유무선헤드폰/블루투스 v5.2/유선연결: 3.5mm/밀폐형/오버이어/음악+통화용/다...,233050
4,젠하이저 모멘텀 4 와이어리스 M4AEBT,유무선헤드폰/블루투스 v5.2/유선연결: 3.5mm/밀폐형/오버이어/음악+통화용/다...,383860


In [4]:
company_list=[]
product_list=[]

for title in data['product_name']:
  splited = title.split(' ',1)
  company_name= splited[0]
  product_name = None
  if len(splited) > 1:
    product_name= splited[1]
  company_list.append(company_name)
  product_list.append(product_name)

In [ ]:
spec_list = data['spec'][0].split('/')
spec_list

In [ ]:
for spec in spec_list:
  if '재생시간' in spec:
    play_time = spec
  elif '무게' in spec:
    weight = spec
  elif '블루투스' in spec:
    bluetooth_version = spec
print(play_time, weight, bluetooth_version)

In [ ]:
play_time_number = play_time.split(' ')[1].strip()
weight_number = weight.split(' ')[1].strip()
bluetooth_version_number = bluetooth_version.split(' ')[1].strip()
print(play_time_number, weight_number, bluetooth_version_number)

In [18]:
category_list=[]
play_time_list=[]
weight_list=[]
bluetooth_version_list=[]
for spec_str in data['spec']:
  spec_list = spec_str.split('/')
  category = spec_list[0]
  category_list.append(category)
  
  play_time_number = None
  weight_number = None
  bluetooth_version_number = None
  for spec in spec_list:
    if '재생시간' in spec:
      play_time_number = spec.split(' ')[1].strip()
    elif '무게' in spec:
      weight_number = spec.split(' ')[1].strip()
    elif '블루투스' in spec:
      if len(spec.split(' '))>1:
        bluetooth_version_number = spec.split(' ')[1].strip()
  play_time_list.append(play_time_number)    
  weight_list.append(weight_number)    
  bluetooth_version_list.append(bluetooth_version_number)  

In [53]:
play_time_list[0:5]
weight_list[0:5]
bluetooth_version_list[0:5]

['24시간', '20시간(ANC', '30시간(ANC', '30시간(ANC', '60시간(ANC']
['252g', '384.8g', '250g', '255g', '293g']
400


In [56]:
import re
def preprocess_play_time(time_str):
  # print('time_str',time_str)
  result = None
  if time_str:
    reg = re.compile('(\d+)(?=시간)')
    result = reg.search(time_str)
 
    return result.group() if result else None
  else:
    return result

print(preprocess_play_time('20-40시간(abc)'))
print(preprocess_play_time('최대 11시간'))

3


[5.0, 5.0, 6.0]

In [57]:
preprocess_play_time_list=[]
for time_str in play_time_list:
  # print('time_str',time_str)
    time = None
    result = preprocess_play_time(time_str)
    # print(result)
    if result:
      time = result
    preprocess_play_time_list.append(time)
  
preprocess_play_time_list[0:10]

400
400
400
[24.0, 20.0, 30.0, 30.0, 60.0, 60.0, 30.0, 29.0, 30.0, 35.0]
[252.0, 384.8, 250.0, 255.0, 293.0, 290.0, 320.0, 220.6, 254.0, 192.0]
[5.3, 5.0, 5.2, 5.2, 5.2, 5.2, 5.2, 5.0, 5.0, 5.2]


In [58]:
data_df = pd.DataFrame()
data_df['category'] = category_list
data_df['company'] = company_list
data_df['product'] = product_list
data_df['price'] = data['price']
data_df['play_time']=preprocess_play_time_list
data_df['weight']=weight_list
data_df['bluetooth_version']=bluetooth_version_list
data_df.head()

,category,company,product,price,play_time,weight,bluetooth_version
0,유무선헤드폰,BOSE,QC 울트라 헤드폰,459000,24.0,252.0,5.3
1,무선헤드폰,APPLE,에어팟 맥스,653070,20.0,384.8,5.0
2,유무선헤드폰,SONY,WH-1000XM5,397790,30.0,250.0,5.2
3,유무선헤드폰,SONY,ULT WEAR WH-ULT900N,233050,30.0,255.0,5.2
4,유무선헤드폰,젠하이저,모멘텀 4 와이어리스 M4AEBT,383860,60.0,293.0,5.2


In [44]:
data_df['category'].value_counts()

[252.0, 384.8, 250.0, 255.0, 293.0, 290.0, 320.0, 220.6, 254.0, 192.0]
[5.3, 5.0, 5.2, 5.2, 5.2, 5.2, 5.2, 5.0, 5.0, 5.2]


In [59]:
data_df_final=data_df[data_df['category'].isin(['유선헤드폰']) == False]
len(data_df_final)

398

In [ ]:
data_df_final['price'].unique()

In [60]:
data_df_final.to_excel('./danawa_data_preprocessing.xlsx', index=False)

In [1]:
danawa_data = pd.read_excel('./danawa_data_preprocessing.xlsx')
danawa_data.head()

,category,company,product,price,play_time,weight,bluetooth_version
0,유무선헤드폰,BOSE,QC 울트라 헤드폰,459000,24.0,252.0,5.3
1,무선헤드폰,APPLE,에어팟 맥스,653070,20.0,384.8,5.0
2,유무선헤드폰,SONY,WH-1000XM5,397790,30.0,250.0,5.2
3,유무선헤드폰,SONY,ULT WEAR WH-ULT900N,233050,30.0,255.0,5.2
4,유무선헤드폰,젠하이저,모멘텀 4 와이어리스 M4AEBT,383860,60.0,293.0,5.2


In [ ]:
play_time_top_list = danawa_data.sort_values(['play_time'],ascending=False)
play_time_top_list

In [ ]:
weight_top_list = danawa_data.sort_values(["weight"],ascending=False)
weight_top_list

In [ ]:
bluetooth_version_top_list = danawa_data.sort_values(['bluetooth_version'],ascending=False)
bluetooth_version_top_list

In [137]:
top_list = danawa_data.sort_values(['play_time','bluetooth_version'],ascending=False)
top_list

,category,company,product,price,play_time,weight,bluetooth_version
257,유무선헤드폰,Picun,F2,33800,100.0,NaN,v5.3
35,유무선헤드폰,마샬,Major V,190000,100.0,186g,LE
212,무선헤드폰,Britz,브리츠인터내셔널 W830BT,122980,95.0,NaN,v4.1
335,유무선헤드폰,HOCO,W35 Max,19900,90.0,262g,v5.3
29,유무선헤드폰,AKG,N9 HYBRID,389900,85.0,281g,v5.3
...,...,...,...,...,...,...,...
325,유무선헤드폰,데논,AH-GC25W,170000,NaN,NaN,NaN
352,유무선헤드폰,몬스터,아이스포츠 BT 프리덤,354100,NaN,220g,NaN
374,유무선헤드폰,베이어다이나믹,AMIRON wireless,990000,NaN,NaN,NaN
389,유무선헤드폰,플랜트로닉스,백비트 505,56200,NaN,155g,NaN


np.float64(28.509383378016086)